In [1]:
import copy
import numpy as np
import torch
from tqdm import tqdm
from sklearn.utils import check_random_state
import pandas as pd
from proplace.clfutils import HiddenPrints
import proplace
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


D:\Apps\anaconda\envs\proplace\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]
[WARNING] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.
 [lazy_loader.py _load]


In [42]:
# load dataset utils
dataset = proplace.InnDataSet("compas")

In [43]:
# get a clf#
torch.manual_seed(101)
np.random.seed(1)
random_state = check_random_state(10)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
d = dataset
torch_model = proplace.train_clf(d.X1_train, d.y1_train, d.X1_test, d.y1_test, 20, epochs=50, data_name="compas", save_clf=False,
                        load_clf=False)
model = proplace.InnModel(dataset, torch_model, 20)

Using cuda device
Epoch 0: train loss: 0.704266369342804
Epoch 0: train loss: 0.6788407564163208
Epoch 0: train loss: 0.6602728366851807
Epoch 0: train loss: 0.6463878154754639
Epoch 0: train loss: 0.6351909041404724
Epoch 0: train loss: 0.5963695645332336
Epoch 0: train loss: 0.5887550711631775
Epoch 0: train loss: 0.6307805776596069
Epoch 0: train loss: 0.53557288646698
Epoch 0: train loss: 0.6204761266708374
Epoch 0: train loss: 0.5046147108078003
Epoch 0: train loss: 0.5312041640281677
Epoch 0: train loss: 0.5049447417259216
Epoch 0: train loss: 0.49821600317955017
Epoch 0: train loss: 0.4659635126590729
Epoch 0: train loss: 0.2855537533760071
Epoch 0: train loss: 0.559812068939209
Epoch 0: train loss: 0.43528714776039124
Epoch 0: train loss: 0.6615065336227417
Epoch 0: train loss: 0.3767836391925812
Epoch 0: train loss: 0.3594193458557129
Epoch 0: train loss: 0.6426411867141724
Epoch 0: train loss: 0.4638318419456482
Epoch 0: train loss: 0.29755380749702454
Epoch 0: train loss: 0.

In [44]:
m2s = proplace.retrain_models(dataset, epochs=50) + proplace.retrain_models_leave_some_out(dataset, epochs=50)

100%|██████████| 10/10 [00:45<00:00,  4.53s/it]


## Prepare for exps

In [45]:
from exputils import *
clf = model
test_xs_vals, test_xs, test_xs_carla, utildataset, nodes = get_test_data(model, m2s, dataset)

## Run Baselines

In [53]:
# ours
rnnce_ces, treer, X_class1_clf_robust = run_rnnce(d, clf, nodes, utildataset, 0.025, test_xs_vals)
proplace_ces = run_proplace(clf, nodes, utildataset, 0.025, treer, X_class1_clf_robust, test_xs_vals)
proplace_scores = eval_ces(clf, m2s, proplace_ces, d, utildataset, test_xs_vals, 0.025)

2635it [01:10, 37.36it/s]
50it [00:41,  1.21it/s]


1.0 >>> validity
1.0 >>> robustness - M2 validity


50it [00:02, 21.45it/s]

1.0 0.006712798616364821 >>> robustness - delta validity, avg bound
0.03892371392318255 >>> l1 cost
0.86 >>> percentage of inliers dataset class 1 points
1.2410574092128752 >>> avg lof dataset class 1 points


In [54]:
wach_ces = run_wachter(clf, test_xs_carla)
wach_scores = eval_ces(clf, m2s, wach_ces, d, utildataset, test_xs_vals,  delta=0.025)

[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Timeout - No Count

50it [00:01, 32.11it/s]

0.0 -0.3819811003183235 >>> robustness - delta validity, avg bound
0.16963035890591982 >>> l1 cost
0.4482758620689655 >>> percentage of inliers dataset class 1 points
1.8088584491928488 >>> avg lof dataset class 1 points


In [55]:
rbr_ces = run_rbr(clf, d, random_state, test_xs_vals)
rbr_scores = eval_ces(clf, m2s, rbr_ces, d, utildataset, test_xs_vals, 0.025)

50it [06:40,  8.01s/it]


1.0 >>> validity
0.9969999999999999 >>> robustness - M2 validity


50it [00:02, 21.09it/s]

0.62 0.03066817466969085 >>> robustness - delta validity, avg bound
0.04319718658904799 >>> l1 cost
0.78 >>> percentage of inliers dataset class 1 points
1.3390591933671456 >>> avg lof dataset class 1 points


In [56]:
robx_ces = run_robx(clf, d, test_xs_vals)
robx_scores = eval_ces(clf, m2s, robx_ces, d, utildataset, test_xs_vals, 0.025)

2635it [00:04, 597.91it/s]


(1822, 7)


50it [00:01, 47.81it/s]


1.0 >>> validity
1.0 >>> robustness - M2 validity


50it [00:02, 19.87it/s]

0.82 0.29518415654362185 >>> robustness - delta validity, avg bound
0.05007915558126085 >>> l1 cost
0.94 >>> percentage of inliers dataset class 1 points
1.1115520645680954 >>> avg lof dataset class 1 points


In [68]:
roar_params = {
        "lr": 0.01,
        "lambda_": 0.01,
        "delta_max": 0.004,
        "norm": 1,
        "t_max_min": 3,
        "loss_type": "MSE",
        "y_target": [1],
        "binary_cat_features": False,
        "loss_threshold": 1e-3,
        "discretize": False,
        "sample": False,
        "lime_seed": 0,
        "seed": 0,
    }
roar_ces = run_roar(clf, test_xs_carla, roar_params)
roar_scores = eval_ces(clf, m2s, roar_ces, d, utildataset, test_xs_vals, delta=0.025)

[INFO] Start generating LIME coefficients [model.py get_counterfactuals]
[INFO] Finished generating LIME coefficients [model.py get_counterfactuals]
0.98 >>> validity
0.9800000000000002 >>> robustness - M2 validity


50it [00:02, 21.44it/s]

1.0 2.2017563019735396 >>> robustness - delta validity, avg bound
0.2186056390734865 >>> l1 cost
0.0 >>> percentage of inliers dataset class 1 points
2.8384434240260816 >>> avg lof dataset class 1 points


In [71]:
protor_ces = run_proto_r(clf, d, utildataset, test_xs_vals, nodes, delta_target=0.025)
protor_scores = eval_ces(clf, m2s, protor_ces, d, utildataset, test_xs_vals, 0.025)

0it [00:00, ?it/s]

[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]


1it [01:03, 63.46s/it]

[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]


2it [02:06, 62.93s/it]

...

[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]


50it [55:07, 66.14s/it]


1.0 >>> validity
1.0 >>> robustness - M2 validity


50it [00:02, 19.31it/s]

1.0 1.2865079344205055 >>> robustness - delta validity, avg bound
0.08393428906575501 >>> l1 cost
0.76 >>> percentage of inliers dataset class 1 points
1.3607431983140341 >>> avg lof dataset class 1 points


In [72]:
milpr_ces = run_milpr(clf, d, nodes, utildataset, 0.03, test_xs_vals)
milpr_scores = eval_ces(clf, m2s, milpr_ces, d, utildataset, test_xs_vals, 0.025)

50it [02:58,  3.58s/it]


1.0 >>> validity
0.998 >>> robustness - M2 validity


50it [00:02, 16.70it/s]

1.0 0.29246658294369426 >>> robustness - delta validity, avg bound
0.040236280590115484 >>> l1 cost
0.68 >>> percentage of inliers dataset class 1 points
1.708005003165761 >>> avg lof dataset class 1 points


In [73]:
# make table
from tabulate import tabulate
scores_names = ["name", "validity", "delta validity", "M2 validity", "l1", "%inlier class 1", "lof class 1"]
scores_table = [scores_names,
                np.concatenate((['wach'], np.round(wach_scores, 3))),
                np.concatenate((['rbr'], np.round(rbr_scores, 3))),
                np.concatenate((['robx'], np.round(robx_scores, 3))),
                np.concatenate((['roar'], np.round(roar_scores, 3))),
                np.concatenate((['proto-r'], np.round(protor_scores, 3))),
                np.concatenate((['milp-r'], np.round(milpr_scores, 3))),
                np.concatenate((['proplace'], np.round(proplace_scores, 3))), ]
print(tabulate(scores_table, headers='firstrow', tablefmt='outline'))

+----------+------------+------------------+---------------+-------+-------------------+---------------+
| name     |   validity |   delta validity |   M2 validity |    l1 |   %inlier class 1 |   lof class 1 |
+==========+============+==================+===============+=======+===================+===============+
| wach     |       0.58 |             0    |         0.566 | 0.17  |             0.448 |         1.809 |
| rbr      |       1    |             0.62 |         0.997 | 0.043 |             0.78  |         1.339 |
| robx     |       1    |             0.82 |         1     | 0.05  |             0.94  |         1.112 |
| roar     |       0.98 |             1    |         0.98  | 0.219 |             0     |         2.838 |
| proto-r  |       1    |             1    |         1     | 0.084 |             0.76  |         1.361 |
| milp-r   |       1    |             1    |         0.998 | 0.04  |             0.68  |         1.708 |
| proplace |       1    |             1    |         1 

In [ ]:
-----+----------